In [ ]:
# データ読み込み
import os
import numpy as np
import pandas as pd

# フォルダ名からコンペ名を取得
compe_name = os.listdir('/kaggle/input')[0]
print("Competition name: " + compe_name)

# データ読み込み
train = pd.read_csv('../input/%s/train.csv' % (compe_name))
test = pd.read_csv('../input/%s/test.csv' % (compe_name))
print("Train: ", train.shape)
print("Test: ", test.shape)

# 提出用ファイルの見本からIndexとObjectiveの列名を取得
sub_sample = pd.read_csv('../input/%s/sample_submission.csv' % (compe_name))
sub_idxcol = sub_sample.columns[0]
sub_objcol = sub_sample.columns[1]

print("Variables: ", train.columns)
print("Submission format: ")
print(sub_sample)
print("Index column name: " + sub_idxcol)
print("Objective column name: " + sub_objcol)

In [ ]:
# trainの型と欠損値を確認
print(train.info())

In [ ]:
# testの型と欠損値を確認
print(train.info())

In [ ]:
# 学習データ・テストデータの準備
obj_var = 'Survived' # train.csvの目的変数の列名を指定する

xtrn = train.drop([obj_var], axis=1)
xtst = test.copy()
ytrn = train[obj_var]

In [ ]:
# 数値データの確認
train.describe()

In [ ]:
# カテゴリーデータの確認
train.describe(include=['O'])

In [ ]:
# 絶対に使用しない変数をリストアップ
drop_vars = ['PassengerId']

# 何らかの理由で使用しない変数をリストに追加
drop_vars += ['Name', 'Sex', 'Age', 'Ticket', 'Fare', 'Cabin', 'Embarked']

# 使用しない変数を削除
xtrn.drop(drop_vars, axis=1, inplace=True)
xtst.drop(drop_vars, axis=1, inplace=True)
print(xtrn)

In [ ]:
# XGBoostモデルの学習
from xgboost import XGBClassifier

model = XGBClassifier(random_state=1)
model.fit(xtrn, ytrn)

In [ ]:
# 学習済みモデルによる予測
from sklearn.metrics import accuracy_score

ptrn = model.predict_proba(xtrn)
ptst = model.predict_proba(xtst)
ltrn = np.argmax(ptrn, axis=1)
ltst = np.argmax(ptst, axis=1)

print(ptrn)
print(ltrn)
print(ytrn)

acctrn = accuracy_score(ytrn, ltrn)
print("XGBoost training accuracy: %f" % (acctrn))

In [ ]:
# 提出用ファイルを作成
csvname = 'submission_xgboost.csv'
print(csvname)
submission = pd.DataFrame({sub_idxcol: sub_sample[sub_idxcol], sub_objcol: ltst})
submission.to_csv(csvname, index=False)
print(submission)